In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [2]:
client = MongoClient("mongodb://localhost:27017")
db = client['new_db']
presentation_collection = db['presentation_news']

In [3]:
presentation_data_list = list(presentation_collection.find())

In [4]:
print(len(presentation_data_list))

182


In [5]:
# presentation_data_list

In [6]:
presentation_data_df = pd.DataFrame(presentation_data_list)

In [7]:
presentation_data_df.head()

,_id,raw_news,processed_text,date,datetime,stock_name,ticker_name,finbert_analysis,stock_data
0,6777a09f8581574bb8d03ac9,Western Carriers stock zooms 13% after securin...,western carrier stock zoom securing r crore co...,2025-01-03,2025-01-03 12:21,Western Carrier,WCIL.NS,"[{'label': 'Positive', 'score': 0.999692678451...","{'avg_price_5d': 114.92250061035156, 'volatili..."
1,6777a0a68581574bb8d03aca,Weekly Tactical Pick | Cello World: This consu...,weekly tactical pick cello world consumer ware...,2025-01-03,2025-01-03 09:26,CELLO,CELLO.NS,"[{'label': 'Positive', 'score': 1.0}]","{'avg_price_5d': 754.4000091552734, 'volatilit..."
2,6777a0d08581574bb8d03acb,Buy Coal India; target of Rs 525: Emkay Global...,buy coal india target r emkay global financial...,2025-01-03,2025-01-03 12:26,Coal India,COALINDIA.NS,"[{'label': 'Positive', 'score': 0.990708470344...","{'avg_price_5d': 385.3499984741211, 'volatilit..."
3,6777a0d38581574bb8d03acc,Pricol shares fall on divestment of wiping bus...,pricol share fall divestment wiping business d...,2025-01-03,2025-01-03 09:58,Pricol,PRICOLLTD.NS,"[{'label': 'Neutral', 'score': 0.9999985694885...","{'avg_price_5d': 547.7749938964844, 'volatilit..."
4,6777a0d78581574bb8d03acd,Shiva Texyarn shares zoom 12% as firm secures ...,shiva texyarn share zoom firm secures major or...,2025-01-03,2025-01-03 13:44,Shiva Texyarn,SHIVATEX.NS,"[{'label': 'Neutral', 'score': 0.9999769926071...","{'avg_price_5d': 222.5474967956543, 'volatilit..."


In [8]:
selected_column_df = presentation_data_df[['finbert_analysis','stock_data']]

In [9]:
selected_column_df.head()

,finbert_analysis,stock_data
0,"[{'label': 'Positive', 'score': 0.999692678451...","{'avg_price_5d': 114.92250061035156, 'volatili..."
1,"[{'label': 'Positive', 'score': 1.0}]","{'avg_price_5d': 754.4000091552734, 'volatilit..."
2,"[{'label': 'Positive', 'score': 0.990708470344...","{'avg_price_5d': 385.3499984741211, 'volatilit..."
3,"[{'label': 'Neutral', 'score': 0.9999985694885...","{'avg_price_5d': 547.7749938964844, 'volatilit..."
4,"[{'label': 'Neutral', 'score': 0.9999769926071...","{'avg_price_5d': 222.5474967956543, 'volatilit..."


In [10]:
def extraction_finbert_data(finbert_list_dict):
    label = finbert_list_dict[0].get('label')    
    score = finbert_list_dict[0].get('score')
    return label, score

In [11]:
selected_column_df[['finbert_label', 'finbert_score']] = selected_column_df['finbert_analysis'].apply(extraction_finbert_data).apply(pd.Series)

C:\Users\AjayPatil\AppData\Local\Temp\ipykernel_19312\2151535678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column_df[['finbert_label', 'finbert_score']] = selected_column_df['finbert_analysis'].apply(extraction_finbert_data).apply(pd.Series)
C:\Users\AjayPatil\AppData\Local\Temp\ipykernel_19312\2151535678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column_df[['finbert_label', 'finbert_score']] = selected_column_df['finbert_analysis'].apply(extraction_finbert_data).apply(p

In [12]:
selected_column_df.head()

,finbert_analysis,stock_data,finbert_label,finbert_score
0,"[{'label': 'Positive', 'score': 0.999692678451...","{'avg_price_5d': 114.92250061035156, 'volatili...",Positive,0.999693
1,"[{'label': 'Positive', 'score': 1.0}]","{'avg_price_5d': 754.4000091552734, 'volatilit...",Positive,1.000000
2,"[{'label': 'Positive', 'score': 0.990708470344...","{'avg_price_5d': 385.3499984741211, 'volatilit...",Positive,0.990708
3,"[{'label': 'Neutral', 'score': 0.9999985694885...","{'avg_price_5d': 547.7749938964844, 'volatilit...",Neutral,0.999999
4,"[{'label': 'Neutral', 'score': 0.9999769926071...","{'avg_price_5d': 222.5474967956543, 'volatilit...",Neutral,0.999977


In [13]:
def extract_stock_data(stock_data):
    avg_price_5d = stock_data.get('avg_price_5d')
    volatility_5d = stock_data.get('volatility_5d')
    avg_volume_5d = stock_data.get('avg_volume_5d')
    open_price_news_day = stock_data.get('open_price_news_day')
    close_price_news_day = stock_data.get('close_price_news_day')
    volume_news_day = stock_data.get('volume_news_day')
    price_movement_1d = stock_data.get('price_movement_1d')
    gap = stock_data.get('gap')
    stock_movement_on_news_day = stock_data.get('stock_movement_on_news_day')
    return avg_price_5d,volatility_5d, avg_volume_5d, open_price_news_day, close_price_news_day, volume_news_day, price_movement_1d,gap, stock_movement_on_news_day

In [14]:
selected_column_df[['avg_price_5d','volatility_5d','avg_volume_5d','open_price_news_day','close_price_news_day','volume_news_day','price_movement_1d','gap','stock_movement_on_news_day']] =selected_column_df['stock_data'].apply(extract_stock_data).apply(pd.Series)

C:\Users\AjayPatil\AppData\Local\Temp\ipykernel_19312\235103632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column_df[['avg_price_5d','volatility_5d','avg_volume_5d','open_price_news_day','close_price_news_day','volume_news_day','price_movement_1d','gap','stock_movement_on_news_day']] =selected_column_df['stock_data'].apply(extract_stock_data).apply(pd.Series)
C:\Users\AjayPatil\AppData\Local\Temp\ipykernel_19312\235103632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_column_

In [15]:
selected_column_df.head()

,finbert_analysis,stock_data,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d,open_price_news_day,close_price_news_day,volume_news_day,price_movement_1d,gap,stock_movement_on_news_day
0,"[{'label': 'Positive', 'score': 0.999692678451...","{'avg_price_5d': 114.92250061035156, 'volatili...",Positive,0.999693,114.922501,0.770297,263441.75,115.989998,121.449997,2463672.0,-1.586341,0.089996,up
1,"[{'label': 'Positive', 'score': 1.0}]","{'avg_price_5d': 754.4000091552734, 'volatilit...",Positive,1.000000,754.400009,10.765754,100198.25,768.000000,769.900024,128644.0,-3.378908,3.400024,up
2,"[{'label': 'Positive', 'score': 0.990708470344...","{'avg_price_5d': 385.3499984741211, 'volatilit...",Positive,0.990708,385.349998,6.213031,10284797.75,393.000000,393.649994,7122319.0,-3.625954,0.049988,up
3,"[{'label': 'Neutral', 'score': 0.9999985694885...","{'avg_price_5d': 547.7749938964844, 'volatilit...",Neutral,0.999999,547.774994,6.341749,587448.00,557.849976,551.049988,494951.0,-1.837412,3.699951,down
4,"[{'label': 'Neutral', 'score': 0.9999769926071...","{'avg_price_5d': 222.5474967956543, 'volatilit...",Neutral,0.999977,222.547497,1.687691,7783.50,228.399994,234.720001,125722.0,-1.838878,5.809998,up


In [16]:
dropped_column_df = selected_column_df.drop(columns=['finbert_analysis','stock_data'])

In [17]:
dropped_column_df.head()

,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d,open_price_news_day,close_price_news_day,volume_news_day,price_movement_1d,gap,stock_movement_on_news_day
0,Positive,0.999693,114.922501,0.770297,263441.75,115.989998,121.449997,2463672.0,-1.586341,0.089996,up
1,Positive,1.000000,754.400009,10.765754,100198.25,768.000000,769.900024,128644.0,-3.378908,3.400024,up
2,Positive,0.990708,385.349998,6.213031,10284797.75,393.000000,393.649994,7122319.0,-3.625954,0.049988,up
3,Neutral,0.999999,547.774994,6.341749,587448.00,557.849976,551.049988,494951.0,-1.837412,3.699951,down
4,Neutral,0.999977,222.547497,1.687691,7783.50,228.399994,234.720001,125722.0,-1.838878,5.809998,up


In [18]:
def convert_label_to_numeric(label):
    if label.strip() == 'Positive':
        return 1
    elif label.strip() == "Neutral":
        return 0
    elif label.strip() == "Negative":
        return -1

In [19]:
dropped_column_df['finbert_label'] = dropped_column_df['finbert_label'].apply(convert_label_to_numeric)

In [20]:
dropped_column_df.head()

,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d,open_price_news_day,close_price_news_day,volume_news_day,price_movement_1d,gap,stock_movement_on_news_day
0,1,0.999693,114.922501,0.770297,263441.75,115.989998,121.449997,2463672.0,-1.586341,0.089996,up
1,1,1.000000,754.400009,10.765754,100198.25,768.000000,769.900024,128644.0,-3.378908,3.400024,up
2,1,0.990708,385.349998,6.213031,10284797.75,393.000000,393.649994,7122319.0,-3.625954,0.049988,up
3,0,0.999999,547.774994,6.341749,587448.00,557.849976,551.049988,494951.0,-1.837412,3.699951,down
4,0,0.999977,222.547497,1.687691,7783.50,228.399994,234.720001,125722.0,-1.838878,5.809998,up


In [21]:
def convert_target_varible(output):
    if output.strip() == "up":
        return 1
    elif output.strip() == "down":
        return 0

In [22]:
dropped_column_df['stock_movement_on_news_day'] = dropped_column_df['stock_movement_on_news_day'].apply(convert_target_varible)

In [23]:
dropped_column_df.head()

,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d,open_price_news_day,close_price_news_day,volume_news_day,price_movement_1d,gap,stock_movement_on_news_day
0,1,0.999693,114.922501,0.770297,263441.75,115.989998,121.449997,2463672.0,-1.586341,0.089996,1
1,1,1.000000,754.400009,10.765754,100198.25,768.000000,769.900024,128644.0,-3.378908,3.400024,1
2,1,0.990708,385.349998,6.213031,10284797.75,393.000000,393.649994,7122319.0,-3.625954,0.049988,1
3,0,0.999999,547.774994,6.341749,587448.00,557.849976,551.049988,494951.0,-1.837412,3.699951,0
4,0,0.999977,222.547497,1.687691,7783.50,228.399994,234.720001,125722.0,-1.838878,5.809998,1


In [24]:
def round_out_value(value):
    return round(value, 3)

In [25]:
dropped_column_df['finbert_score'] = dropped_column_df['finbert_score'].apply(round_out_value)
dropped_column_df['avg_price_5d'] = dropped_column_df['avg_price_5d'].apply(round_out_value)
dropped_column_df['volatility_5d'] = dropped_column_df['volatility_5d'].apply(round_out_value)
dropped_column_df['avg_volume_5d'] = dropped_column_df['avg_volume_5d'].apply(round_out_value)
dropped_column_df['open_price_news_day'] = dropped_column_df['open_price_news_day'].apply(round_out_value)
dropped_column_df['close_price_news_day'] = dropped_column_df['close_price_news_day'].apply(round_out_value)
dropped_column_df['volume_news_day'] = dropped_column_df['volume_news_day'].apply(round_out_value)
dropped_column_df['price_movement_1d'] = dropped_column_df['price_movement_1d'].apply(round_out_value)
dropped_column_df['gap'] = dropped_column_df['gap'].apply(round_out_value)

In [26]:
dropped_column_df.head()

,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d,open_price_news_day,close_price_news_day,volume_news_day,price_movement_1d,gap,stock_movement_on_news_day
0,1,1.000,114.923,0.770,263441.75,115.99,121.45,2463672.0,-1.586,0.09,1
1,1,1.000,754.400,10.766,100198.25,768.00,769.90,128644.0,-3.379,3.40,1
2,1,0.991,385.350,6.213,10284797.75,393.00,393.65,7122319.0,-3.626,0.05,1
3,0,1.000,547.775,6.342,587448.00,557.85,551.05,494951.0,-1.837,3.70,0
4,0,1.000,222.547,1.688,7783.50,228.40,234.72,125722.0,-1.839,5.81,1


In [27]:
dropped_column_df = dropped_column_df.fillna(0)


RANDOM FOREST APPLY

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [44]:
x = dropped_column_df[['finbert_label','finbert_score','avg_price_5d','volatility_5d','avg_volume_5d',]]

In [45]:
y = dropped_column_df['open_price_news_day']

In [46]:
x.head()

,finbert_label,finbert_score,avg_price_5d,volatility_5d,avg_volume_5d
0,1,1.000,114.923,0.770,263441.75
1,1,1.000,754.400,10.766,100198.25
2,1,0.991,385.350,6.213,10284797.75
3,0,1.000,547.775,6.342,587448.00
4,0,1.000,222.547,1.688,7783.50


In [47]:
y.head()

0    115.99
1    768.00
2    393.00
3    557.85
4    228.40
Name: open_price_news_day, dtype: float64

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state= 42)

In [68]:
random_regressor = RandomForestRegressor()
random_regressor.fit(x_train, y_train)

RandomForestRegressor()

In [70]:
y_predict = random_regressor.predict(x_test)

In [71]:
print("Mean Absolute Error:", mean_absolute_error(y_test, y_predict))
print("Mean Squared Error:", mean_squared_error(y_test, y_predict))
print("R^2 Score:", r2_score(y_test, y_predict))

Mean Absolute Error: 94.00129036363627
Mean Squared Error: 37527.32726934458
R^2 Score: 0.9949923962800357


In [73]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [74]:
grid_search = GridSearchCV(
    estimator=random_regressor,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='neg_mean_squared_error',
)

In [75]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error', verbose=2)

In [76]:
grid_search.best_score_

-260358.7594817346

In [77]:
grid_search.best_params_

{'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [78]:
y_predict = grid_search.predict(x_test)

In [79]:
print("Mean Absolute Error:", mean_absolute_error(y_test, y_predict))
print("Mean Squared Error:", mean_squared_error(y_test, y_predict))
print("R^2 Score:", r2_score(y_test, y_predict))

Mean Absolute Error: 206.041532363636
Mean Squared Error: 110191.68134891712
R^2 Score: 0.9852961478052631


In [80]:
import dill
with open("random_regressor.pkl", "wb") as file:
    dill.dump(grid_search.best_estimator_, file)